In [6]:
# Import necessary libraries
import os
import pandas as pd
from astropy.coordinates import SkyCoord
from astroquery.mast import Catalogs
from astropy import units as u
from tqdm import tqdm
import glob
import requests

In [ ]:
# Step 1.1: Extract SDSS DR18 LRG Samples using CasJobs
# Data listed in dr18_lrg_sample.csv (Sample Size: 1,174,900)
# Criteria : 

# SELECT
  # s.specobjid,
  # s.z AS redshift,                   
  # s.veldisp,
  # p.ra, p.dec, p.u, p.g, p.r, p.i, p.modelMag_r,
  # s.programname, s.plate, s.fiberid, s.mjd into mydb.LRG_full_catalog from SpecObjAll AS s
# JOIN PhotoObjAll AS p ON s.bestobjid = p.objid
# WHERE
  # s.class = 'GALAXY'
  # AND s.z BETWEEN 0.1 AND 0.7
  # AND s.veldisp > 0 AND s.veldisp < 500
  # AND s.programname IN ('boss', 'eboss')
  # AND (p.r - p.i) > 0.5
  # AND (p.g - p.r) > 0.7
  # AND p.modelMag_r BETWEEN 16 AND 21

In [ ]:
BATCH_SIZE = 10000
INPUT_CSV = "sdss_dr18_lrg_sample.csv"
OUTPUT_CSV_TEMPLATE = "sdss_lrg_ps1_matched_batch_{batch_num}.csv"
OUTPUT_FOLDER = "sdss_lrg_queried_objects"

df = pd.read_csv(INPUT_CSV)
n_objects = len(df)

def query_ps1_batch(batch_df):
    g_mag, r_mag, i_mag = [], [], []
    g_depth, r_depth, i_depth = [], [], []
    g_fwhm, r_fwhm, i_fwhm = [], [], []

    for _, row in tqdm(batch_df.iterrows(), total=len(batch_df)):
        try:
            coord = SkyCoord(ra=row['ra'], dec=row['dec'], unit='deg', frame='icrs')
            result = Catalogs.query_region(coord, radius=2.5 * u.arcsec, catalog='PanSTARRS', data_release='dr2')
            if len(result) > 0:
                best = result[0]
                g_mag.append(best.get('gMeanPSFMag'))
                r_mag.append(best.get('rMeanPSFMag'))
                i_mag.append(best.get('iMeanPSFMag'))

                g_depth.append(best.get('gMeanDepth'))
                r_depth.append(best.get('rMeanDepth'))
                i_depth.append(best.get('iMeanDepth'))

                g_fwhm.append(best.get('gFWHM'))
                r_fwhm.append(best.get('rFWHM'))
                i_fwhm.append(best.get('iFWHM'))
            else:
                g_mag.append(None)
                r_mag.append(None)
                i_mag.append(None)
                g_depth.append(None)
                r_depth.append(None)
                i_depth.append(None)
                g_fwhm.append(None)
                r_fwhm.append(None)
                i_fwhm.append(None)
        except Exception:
            g_mag.append(None)
            r_mag.append(None)
            i_mag.append(None)
            g_depth.append(None)
            r_depth.append(None)
            i_depth.append(None)
            g_fwhm.append(None)
            r_fwhm.append(None)
            i_fwhm.append(None)

    batch_df['ps1_g_mag'] = g_mag
    batch_df['ps1_r_mag'] = r_mag
    batch_df['ps1_i_mag'] = i_mag
    batch_df['ps1_g_depth'] = g_depth
    batch_df['ps1_r_depth'] = r_depth
    batch_df['ps1_i_depth'] = i_depth
    batch_df['ps1_g_fwhm'] = g_fwhm
    batch_df['ps1_r_fwhm'] = r_fwhm
    batch_df['ps1_i_fwhm'] = i_fwhm

    # Filter: keep only rows with at least 2 non-null PS1 magnitudes
    mask = (
        batch_df[['ps1_g_mag', 'ps1_r_mag', 'ps1_i_mag']]
        .notnull()
        .sum(axis=1) >= 2
    )
    filtered_df = batch_df[mask].reset_index(drop=True)
    print(f"Filtered from {len(batch_df)} → {len(filtered_df)} rows with ≥ 2 PS1 bands")
    return filtered_df

for i in range(0, n_objects, BATCH_SIZE):
    batch_num = i // BATCH_SIZE + 1
    output_file = os.path.join(OUTPUT_FOLDER, OUTPUT_CSV_TEMPLATE.format(batch_num=batch_num))
    
    if os.path.exists(output_file):
        print(f"Batch {batch_num} already done (found {output_file}), skipping...")
        continue

    batch_df = df.iloc[i:i+BATCH_SIZE].copy()
    print(f"\nProcessing batch {batch_num} ({i} to {i + len(batch_df) - 1})...")
    filtered_df = query_ps1_batch(batch_df)
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    filtered_df.to_csv(output_file, index=False)
    print(f"Saved filtered batch {batch_num} to {output_file}")

print("\nAll batches processed!")

In [10]:
INPUT_FOLDER = "sdss_lrg_queried_objects"
CUTOUT_LIST_CSV = "qualified_cutout_targets.csv"

def gather_cutout_targets(input_folder, output_csv):
    cutout_rows = []

    for filename in sorted(os.listdir(input_folder)):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            print(f"Processing {filename}...")
            df = pd.read_csv(file_path)

            # Convert "--" strings to NaN explicitly if necessary
            for col in ['ps1_g_mag', 'ps1_r_mag', 'ps1_i_mag']:
                df[col] = pd.to_numeric(df[col], errors='coerce')

            # Count how many PS1 magnitudes are valid (not NaN)
            df['valid_ps1_mags'] = df[['ps1_g_mag', 'ps1_r_mag', 'ps1_i_mag']].notnull().sum(axis=1)

            # Keep only rows with ≥ 2 valid PS1 bands
            qualified = df[df['valid_ps1_mags'] >= 2].copy()

            # Optional: record which bands are valid
            qualified['has_g'] = qualified['ps1_g_mag'].notnull()
            qualified['has_r'] = qualified['ps1_r_mag'].notnull()
            qualified['has_i'] = qualified['ps1_i_mag'].notnull()

            # Append essential info for cutout download
            cutout_rows.append(qualified[['specobjid', 'ra', 'dec', 'has_g', 'has_r', 'has_i']])

    # Combine and save
    all_cutouts = pd.concat(cutout_rows, ignore_index=True)
    all_cutouts.to_csv(output_csv, index=False)
    print(f"\nSaved {len(all_cutouts)} qualified cutout targets to {output_csv}")

    # Fix: avoid scientific notation by converting to string
    all_cutouts['specobjid'] = all_cutouts['specobjid'].astype(str)
    
    all_cutouts.to_csv(output_csv, index=False)

gather_cutout_targets("sdss_lrg_queried_objects", "qualified_cutout_targets.csv")

Processing sdss_lrg_ps1_matched_batch_1.csv...
Processing sdss_lrg_ps1_matched_batch_10.csv...
Processing sdss_lrg_ps1_matched_batch_100.csv...
Processing sdss_lrg_ps1_matched_batch_101.csv...
Processing sdss_lrg_ps1_matched_batch_102.csv...
Processing sdss_lrg_ps1_matched_batch_103.csv...
Processing sdss_lrg_ps1_matched_batch_104.csv...
Processing sdss_lrg_ps1_matched_batch_105.csv...
Processing sdss_lrg_ps1_matched_batch_106.csv...
Processing sdss_lrg_ps1_matched_batch_107.csv...
Processing sdss_lrg_ps1_matched_batch_108.csv...
Processing sdss_lrg_ps1_matched_batch_109.csv...
Processing sdss_lrg_ps1_matched_batch_11.csv...
Processing sdss_lrg_ps1_matched_batch_110.csv...
Processing sdss_lrg_ps1_matched_batch_111.csv...
Processing sdss_lrg_ps1_matched_batch_112.csv...
Processing sdss_lrg_ps1_matched_batch_113.csv...
Processing sdss_lrg_ps1_matched_batch_114.csv...
Processing sdss_lrg_ps1_matched_batch_115.csv...
Processing sdss_lrg_ps1_matched_batch_116.csv...
Processing sdss_lrg_ps1_